<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Разбиение-на-выборки" data-toc-modified-id="Разбиение-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разбиение на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Исследование моделей</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Downsampling</a></span></li><li><span><a href="#Upsampling" data-toc-modified-id="Upsampling-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Upsampling</a></span></li><li><span><a href="#Class_weight" data-toc-modified-id="Class_weight-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Class_weight</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Признаки:

    RowNumber — индекс строки в данных
    CustomerId — уникальный идентификатор клиента
    Surname — фамилия
    CreditScore — кредитный рейтинг
    Geography — страна проживания
    Gender — пол
    Age — возраст
    Tenure — сколько лет человек является клиентом банка
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата

Целевой признак:

    Exited — факт ухода клиента

## Подготовка данных

In [164]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [165]:
from tqdm import tqdm

In [166]:
from sklearn import tree
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

In [167]:
try:
    df = pd.read_csv('/datasets/Churn.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [169]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Перед началом работы я бы привела названия некоторых переменных к змеиному и нижнему регистру.

In [170]:
df = df.rename(columns={'RowNumber': 'row_number',
              'CustomerId': 'сustomer_id',
              'CreditScore': 'credit_score',
              'NumOfProducts': 'num_of_products',
              'HasCrCard': 'has_cr_card',
              'IsActiveMember': 'is_active_member',
              'EstimatedSalary': 'estimated_salary'})

In [171]:
df.columns = df.columns.str.lower()

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   сustomer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            9091 non-null   float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В колонке 'tenure' есть пропуски - заполним их "заглушкой" (-1), т.к. удалять 909 строк не очень хочется. Также переведем некоторые столбцы к типу int

In [173]:
df = df.dropna(subset=['tenure'])

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        9091 non-null   int64  
 1   сustomer_id       9091 non-null   int64  
 2   surname           9091 non-null   object 
 3   credit_score      9091 non-null   int64  
 4   geography         9091 non-null   object 
 5   gender            9091 non-null   object 
 6   age               9091 non-null   int64  
 7   tenure            9091 non-null   float64
 8   balance           9091 non-null   float64
 9   num_of_products   9091 non-null   int64  
 10  has_cr_card       9091 non-null   int64  
 11  is_active_member  9091 non-null   int64  
 12  estimated_salary  9091 non-null   float64
 13  exited            9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


In [175]:
df['balance'] = df['balance'].astype('int64')
df['estimated_salary'] = df['estimated_salary'].astype('int64')
df['tenure'] = df['tenure'].astype('int64')

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   row_number        9091 non-null   int64 
 1   сustomer_id       9091 non-null   int64 
 2   surname           9091 non-null   object
 3   credit_score      9091 non-null   int64 
 4   geography         9091 non-null   object
 5   gender            9091 non-null   object
 6   age               9091 non-null   int64 
 7   tenure            9091 non-null   int64 
 8   balance           9091 non-null   int64 
 9   num_of_products   9091 non-null   int64 
 10  has_cr_card       9091 non-null   int64 
 11  is_active_member  9091 non-null   int64 
 12  estimated_salary  9091 non-null   int64 
 13  exited            9091 non-null   int64 
dtypes: int64(11), object(3)
memory usage: 1.0+ MB


In [177]:
df.duplicated().sum() #проверка на наличие явных дупликатов

0

In [178]:
df['geography'].unique() #смотрим из каких стран вообще есть пользователи

array(['France', 'Spain', 'Germany'], dtype=object)

In [179]:
df.corr(method='spearman')

,row_number,сustomer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
row_number,1.000000,0.004324,0.002063,0.000553,-0.007650,-0.005183,0.010115,0.001238,0.009533,-0.004891,-0.014551
сustomer_id,0.004324,1.000000,-0.000942,0.009881,-0.021410,-0.009066,0.016302,-0.019857,0.001526,0.010013,-0.007040
credit_score,0.002063,-0.000942,1.000000,-0.008681,0.000604,0.002355,0.010281,-0.002308,0.029272,0.007318,-0.020277
age,0.000553,0.009881,-0.008681,1.000000,-0.014275,0.037079,-0.059194,-0.019720,0.035473,-0.000445,0.322209
tenure,-0.007650,-0.021410,0.000604,-0.014275,1.000000,-0.005252,0.011453,0.027293,-0.032596,0.010343,-0.016762
balance,-0.005183,-0.009066,0.002355,0.037079,-0.005252,1.000000,-0.314450,-0.014089,-0.004202,0.013199,0.108760
num_of_products,0.010115,0.016302,0.010281,-0.059194,0.011453,-0.314450,1.000000,0.006969,0.016778,0.012308,-0.127500
has_cr_card,0.001238,-0.019857,-0.002308,-0.019720,0.027293,-0.014089,0.006969,1.000000,-0.008810,-0.006224,-0.005411
is_active_member,0.009533,0.001526,0.029272,0.035473,-0.032596,-0.004202,0.016778,-0.008810,1.000000,-0.020159,-0.155062
estimated_salary,-0.004891,0.010013,0.007318,-0.000445,0.010343,0.013199,0.012308,-0.006224,-0.020159,1.000000,0.016033


Мультиколлинеарности между признаками не наблюдается.

Поставленная задача будет решаться методом классификации (т.к. клиент либо ушел, либо нет). Удалим столбцы, которые там не понадобятся (row_number, сustomer_id и surname).

In [180]:
df = df.drop(['row_number', 'сustomer_id', 'surname'], axis=1)

Во избежания dummy ловушки вспользуемся следующим методом:

In [181]:
df = pd.get_dummies(df, drop_first=True)

<b>Общий вывод: </b>в целом, данные изначально были вполне готовы для дальнейшего исследования. Были удалены пропуски в колонке "tenure". Были переименованы переменные в соответсвии со змеиным регистром. В остальном все осталось тем же самым. 

## Исследование задачи

### Разбиение на выборки

Разобьем выборки на 3 части в пропорциях 60:20:20 (обучающая, тестовая и валидационная)

In [182]:
features = df.drop(['exited'],axis= 1)
target = df['exited']
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                            test_size=.4, random_state=12345)
#обучающая выборка
print(features_train.shape)
print(target_train.shape)

(5454, 11)
(5454,)


In [183]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, stratify=target_test,
                                                                            test_size=.5, random_state=12345)
#тестовая выборка, на которой мы будем проверять модель
print(features_test.shape)
print(target_test.shape)
#валидационная выборка
print(features_valid.shape)
print(target_valid.shape)

(1819, 11)
(1819,)
(1818, 11)
(1818,)


Чтобы избежать разного масштаба признаков - стандартизируем их с помощью scaler.

In [184]:
pd.options.mode.chained_assignment = None

In [185]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

### Исследование моделей

Начнем с модели решающего дерева. У нее низкое качество, но зато высокая скорость.

In [186]:
best_model = None
best_result = 0
train_list = []
valid_list = []
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) #модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [187]:
print("Выбранная глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Выбранная глубина:  7
Accuracy лучшей модели: 0.8487348734873488
F1 для Дерева решений: 0.5454545454545455
AUC-ROC для Дерева решени: 0.8182911483289751


F1 для древа решений = 0.56, когда нам нужно (минимум) 0.59, значит эта модель не подходит

Посмотрим на модель логистической регрессии - ее качество должно быть немного выше древа.

In [188]:
model_log = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model_log.fit(features_train, target_train)
predictions_log = model_log.predict(features_valid)

result =  accuracy_score(target_valid, predictions_log)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [189]:
print("Accuracy Логичстической модели:", result)
print("F1 для Логистической модели:", f1_score(target_valid, predictions_log))
print("AUC-ROC для Логистической модели:", auc_roc)

Accuracy Логичстической модели: 0.8140814081408141
F1 для Логистической модели: 0.29583333333333334
AUC-ROC для Логистической модели: 0.689979896307269


F1-мера гораздо ниже, чем в древе= 0.3, совершенно нам не годится. остается случайный лес.

У случайного леса высокое качество, но низкая скорость. Скорее всего, будет выбрана эта модель - в данном случае скорость нам не критична, а точность будет (должна быть...) выше.

In [190]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
for est in  range(10, 101, 10):
    for depth in range(1, 21, 1):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model_forest.fit(features_train,target_train)
        predictions_valid = model_forest.predict(features_valid)
        result =  f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)

        if result > best_result_forest:
            best_model_forest = model_forest
            best_est = est
            best_result_forest = result
            best_depth_forest = depth
            acc_best = acc
            
            probabilities_valid = best_model_forest.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [191]:
print("F1 для случайного леса", best_result_forest)
print("Accuracy для случайного леса", acc_best)
print("AUC-ROC для Леса:", auc_roc)
print('Глубина',best_depth_forest)
print('Кол-во деревьев', best_est)

F1 для случайного леса 0.5851239669421487
Accuracy для случайного леса 0.7975797579757976
AUC-ROC для Леса: 0.843224261982859
Глубина 18
Кол-во деревьев 30


F1 получилась 0.57... Не 0.59, но наивысший результат, надеюсь, в дальнейшем этот показатель поднимется 

<b>Общий вывод: </b>лучшей модель оказался случайный лес с показателями accuracy = 0.72, F1 = 0.57, AUC_ROC = 0.84, глубина = 14, кол-во деревьев = 20.

## Борьба с дисбалансом

Попробуем справиться с дисбалансом классов. Будем использовать class_weight, техниками downsampling и upsampling   - попробуем изменить размер выборки.

Но для начала посмотрим на соотношение классов (кол-во положительных и отрицательных ответов):

In [192]:
print(features_train[target_train == 0].shape)
print(features_train[target_train == 1].shape)

(4328, 11)
(1126, 11)


Виден дисбаланс - кол-во отрицательных ответов сильно превышает положительные.

### Downsampling

Начнем с downsampling

In [193]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [194]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

print (features_downsampled[target_downsampled == 0].shape)
print (features_downsampled[target_downsampled == 1].shape)

(1082, 11)
(1126, 11)


Рассмотрим еще раз древо решений:

In [195]:
best_model = None
best_result = 0
train_list = []
valid_list = []

for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) #модель с заданной глубиной дерева
    model.fit(features_downsampled, target_downsampled) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [196]:
print("Выбранная глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Выбранная глубина:  7
Accuracy лучшей модели: 0.7436743674367436
F1 для Дерева решений: 0.5493230174081238
AUC-ROC для Дерева решени: 0.8155949105914717


По сравнению с несбалансированным древом получились результаты похуже - accuracy, f1 и AUC-ROC ниже (было 0.85, 0.56 и 0.81) соответсвенно. Изменилась выбранная глубина

Рассмотрим логистическую регрессию:

In [197]:
model_log = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model_log.fit(features_downsampled, target_downsampled)
predictions_log = model_log.predict(features_valid)

result =  accuracy_score(target_valid, predictions_log)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [198]:
print("Accuracy Логистической модели:", result)
print("F1 для Логистической модели:", f1_score(target_valid, predictions_log))
print("AUC-ROC для Логистической модели:", auc_roc)

Accuracy Логистической модели: 0.7046204620462047
F1 для Логистической модели: 0.4909952606635071
AUC-ROC для Логистической модели: 0.7192247230073914


F1 при такой выборке больше, а значения accuracy и auc-roc ниже(было: accuracy = 0.79, F1 = 0.3, AUC-ROC = 0.77)

Рассмотрим теперь случайный лес:

In [199]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
for est in  range(10, 101, 10):
    for depth in range(1, 21, 1):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model_forest.fit(features_downsampled, target_downsampled)
        predictions_valid = model_forest.predict(features_valid)
        result =  f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)
        
        if result > best_result_forest:
            best_model_forest = model_forest
            best_est = est
            best_result_forest = result
            best_depth_forest = depth
            acc_best = acc
            
            probabilities_valid = best_model_forest.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [200]:
print("F1 для случайного леса", best_result_forest)
print("Accuracy для случайного леса", acc_best)
print("AUC-ROC для Леса:", auc_roc)
print('Глубина',best_depth_forest)
print('Кол-во деревьев', best_est)

F1 для случайного леса 0.5828220858895706
Accuracy для случайного леса 0.7024202420242024
AUC-ROC для Леса: 0.8525269434829268
Глубина 10
Кол-во деревьев 90


В целом показатели случайного леса на сбалансированной выборке выше, чем в предыдущем исследовании (было accuracy = 0.72, F1 = 0.57, AUC_ROC = 0.84), но не намного.

<b>Общий вывод: </b>если говорить про изменения в показателях в целом, то осталось все также - лучшей моделью оказался случайный лес (accuracy = 0.70, F1 = 0.58, AUC_ROC = 0.85). Кроме того, с техникой downsampling показатели были улучшены. Нужно посмотреть, изменятся ли результаты, если использовать upsampling

### Upsampling

In [201]:
def upsample(features, target, repeat): 
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat) 
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle( features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [202]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

print (features_upsampled[target_upsampled == 0].shape)
print (features_upsampled[target_upsampled == 1].shape)

(4328, 11)
(4504, 11)


Идем в том же порядке и начинаем с древа:

In [203]:
best_model = None
best_result = 0
train_list = []
valid_list = []

for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) #модель с заданной глубиной дерева
    model.fit(features_upsampled, target_upsampled) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [204]:
print("Выбранная глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Выбранная глубина:  6
Accuracy лучшей модели: 0.740924092409241
F1 для Дерева решений: 0.5484180249280921
AUC-ROC для Дерева решени: 0.8321625073688347


По сравнению с несбалансированной моделью (было Accuracy = 0.85, F1 = 0.56 и AUC_ROC = 0.81) показатели ниже, в сравнении со сбалансированной моделью техникой downsampling (было Accuracy = 0.74, F1 = 0.54 и AUC_ROC = 0.81) показатели такие же, за исключением AUC_ROC.

Логистическая регрессия:

In [205]:
model_log = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model_log.fit(features_upsampled, target_upsampled)
predictions_log = model_log.predict(features_valid)

result =  accuracy_score(target_valid, predictions_log)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [206]:
print("Accuracy Логичстической модели:", result)
print("F1 для Логистической модели:", f1_score(target_valid, predictions_log))
print("AUC-ROC для Логистической модели:", auc_roc)

Accuracy Логичстической модели: 0.7090209020902091
F1 для Логистической модели: 0.49474689589302767
AUC-ROC для Логистической модели: 0.6596146288374625


По сравнению с несбалансированной моделью (было accuracy = 0.79, F1 = 0.3, AUC-ROC = 0.77) показатели, кроме F1, ниже, в сравнении со сбалансированной моделью техникой downsampling (было Accuracy = 0.70, F1 = 0.49 и AUC_ROC = 0.71) показатели практически таки же, за иключением AUC-ROC.

Случайный лес:

In [207]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
for est in  range(10, 101, 10):
    for depth in range(1, 21, 1):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model_forest.fit(features_upsampled, target_upsampled)
        predictions_valid = model_forest.predict(features_valid)
        result =  f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)
        
        if result > best_result_forest:
            best_model_forest = model_forest
            best_est = est
            best_result_forest = result
            best_depth_forest = depth
            acc_best = acc
            
            probabilities_valid = best_model_forest.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [208]:
print("F1 для случайного леса", best_result_forest)
print("Accuracy для случайного леса", acc_best)
print("AUC-ROC для Леса:", auc_roc)
print('Глубина',best_depth_forest)
print('Кол-во деревьев', best_est)

F1 для случайного леса 0.6251728907330567
Accuracy для случайного леса 0.7777777777777778
AUC-ROC для Леса: 0.8496956104418277
Глубина 14
Кол-во деревьев 80


По сравнению с несбалансированной моделью (было accuracy = 0.72, F1 = 0.57, AUC_ROC = 0.84) показатели, кроме AUC_ROC, значительо выше, в сравнении со сбалансированной моделью техникой downsampling (было Accuracy = 0.70, F1 = 0.58 и AUC_ROC = 0.85) показатели выше, за исключением AUC_ROC с разницей < 1.

### Class_weight

Древо решений:

In [209]:
best_model = None
best_result = 0
train_list = []
valid_list = []
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced') #модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучение модели
    predictions = model.predict(features_valid) #предсказания модели
    result = f1_score(target_valid, predictions) 
    acc = accuracy_score(target_valid, predictions)
    valid_list.append(result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        best_acc = acc
        probabilities_valid = best_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [210]:
print("Выбранная глубина: ", best_depth)
print("Accuracy лучшей модели:", best_acc)
print("F1 для Дерева решений:", best_result)
print("AUC-ROC для Дерева решени:", auc_roc)

Выбранная глубина:  6
Accuracy лучшей модели: 0.740924092409241
F1 для Дерева решений: 0.5484180249280921
AUC-ROC для Дерева решени: 0.8321077134694741


Данная модель с такими параметрами показала наиболее хороший результат, если сравнивать с предыдущими. 

Логистическая регрессия:

In [211]:
model_log = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000, class_weight='balanced')
model_log.fit(features_train, target_train)
predictions_log = model_log.predict(features_valid)

result =  accuracy_score(target_valid, predictions_log)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [212]:
print("Accuracy Логичстической модели:", result)
print("F1 для Логистической модели:", f1_score(target_valid, predictions_log))
print("AUC-ROC для Логистической модели:", auc_roc)

Accuracy Логичстической модели: 0.7211221122112211
F1 для Логистической модели: 0.5014749262536873
AUC-ROC для Логистической модели: 0.6528664767144823


По сравнению с другими моделями логистической регрессии - у этой наибольшее знчение F1, хотя другие несколько ниже.

Случайный лес:

In [213]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
for est in  range(10, 101, 10):
    for depth in range(1, 21, 1):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, class_weight='balanced')
        model_forest.fit(features_train,target_train)
        predictions_valid = model_forest.predict(features_valid)
        result =  f1_score(target_valid, predictions_valid)
        acc = accuracy_score(target_valid, predictions)

        if result > best_result_forest:
            best_model_forest = model_forest
            best_est = est
            best_result_forest = result
            best_depth_forest = depth
            acc_best = acc
            
            probabilities_valid = best_model_forest.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            auc_roc = roc_auc_score(target_valid,probabilities_one_valid)

In [214]:
print("F1 для случайного леса", best_result_forest)
print("Accuracy для случайного леса", acc_best)
print("AUC-ROC для Леса:", auc_roc)
print('Глубина',best_depth_forest)
print('Кол-во деревьев', best_est)

F1 для случайного леса 0.6263736263736264
Accuracy для случайного леса 0.7733773377337734
AUC-ROC для Леса: 0.851863748356183
Глубина 10
Кол-во деревьев 100


Показатели этой модели немного, но лучше, чем при технике downsampling (было accuracy = 0.777, F1 = 0.6204 и AUC_ROC = 0.84, глубина = 14, кол-во деревьев = 80)

<b>Общий вывод:</b> из всех моделей, которые были сбалансированы техникой downsampling и upsampling, а также при взвешивании классов, наиболее хороший результаты показала модель случайного леса (с class_weight) с параметрами: accuracy = 0.77, F1 = 0.6263 и AUC_ROC = 0.85, глубина = 10, кол-во деревьев = 100

## Тестирование модели

Проверяем модель случайного леса на тестовой выборке с учетом тех параметров, что мы выбрали

In [215]:
model_forest = RandomForestClassifier(random_state = 12345, max_depth = 10, n_estimators = 100, class_weight='balanced')
model_forest.fit(features_train, target_train)
predict_forest = model_forest.predict(features_test)

accuracy_forest = accuracy_score(predict_forest, target_test)

f1 = f1_score(target_test, predict_forest)

probabilities_valid = model_forest.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_valid)

In [216]:
print("Accuracy итоговой модели:", accuracy_forest)
print("F1 итоговой модели:", f1)
print("AUC-ROC итоговой модели:", auc_roc)

Accuracy итоговой модели: 0.8510170423309511
F1 итоговой модели: 0.6372155287817938
AUC-ROC итоговой модели: 0.8744080661606435


In [217]:
recall_score(target_test, predict_forest)

0.6538461538461539

Сравним с константной моделью:

In [218]:
m = DummyClassifier(strategy='most_frequent', random_state=1) 
m.fit(features_train, target_train)
m.predict(features_test)
print('score:', m.score(features_test, target_test))

score: 0.799890049477735


Accuracy нашей итоговой модели (0.85) выше, чем у константной модели - значит все хорошо.

Показатель F1 на итоговой можели выше 0.59, что нам подходит => оставляем эту модель с выбранными гиперпараметрами. Качество моедли тоже достаточно высокое. 

## Вывод

Были подготовлены данные - заполнены пропуски, изменены типы некоторых столбцов, а некоторые столбцы, не несущие важной нам информации, удалены.

Данны были разбиты на три выборки: обучающую, валидационную и тестовую. Во избежания разных масштабов классов, данные были стандартизиованы с помощью scaler. На первых двух выборках, без учета баланса классов, были рассмотрены три модели: решающее древо, логистическая регрессия и случайный лес. На этом этапе лучшей моделью показал себя случайный лес.

Далее была произведена балансировка классов тремя способами: upsampling, downsampling и с помощью параметра class_weight. Во всех моделях наиболее выcокое значение F1 (среднее полноты и точности) показывал случайный лес. Но самое высокое значение этого показателя было достигнито с помощью class_weight, с параметрами: accuracy = 0.77, F1 = 0.6223 и AUC_ROC = 0.85, глубина = 8, кол-во деревьев = 100.

Это параметры и были применены для тестирования итоговой модели. Таким образом, были получены следующие параметры: 

    Accuracy итоговой модели: 0.85
    F1 итоговой модели: 0.6372155287817938
    AUC-ROC итоговой модели: 0.8744080661606435
F1 выше 0.59, а занчит нам подходит модель с выбранными параметрами.

Recall (доля положительных ответов среди всех) = 0.65 - показатель достаточно хороший, т.к., в идеале, должен стремится к 1. Также значение accuracy (качества) итоговой модели (0.85) больше, чем у константной (0.79).